# Pegasus Pseudobulk Analysis

Author: [Bo Li](https://github.com/bli25), [Yiming Yang](https://github.com/yihming)<br />
Date: 2022-03-09 <br />
Notebook Source: [pseudobulk.ipynb](https://raw.githubusercontent.com/lilab-bcb/pegasus-tutorials/main/notebooks/pseudobulk.ipynb)

In [ ]:
import pegasus as pg
import pandas as pd

## Dataset

In this tutorial, we'll use a gene-count matrix dataset on human bone marrow from 8 donors, create pseudobulk matrix regarding donors, and perform pseudobulk analysis on the data.

The dataset is stored at https://storage.googleapis.com/terra-featured-workspaces/Cumulus/MantonBM_nonmix_subset.zarr.zip. You can also use [gsutil](https://cloud.google.com/storage/docs/gsutil) to download it via its Google bucket URL (gs://terra-featured-workspaces/Cumulus/MantonBM_nonmix_subset.zarr.zip).

Now load the count matrix:

In [ ]:
data = pg.read_input('MantonBM_result.zarr.zip')

## Generate pseudobulk matrix

Pegasus provides `pseudobulk` function to generate pseudobulk matrix. The code below generate a pseudobulk count matrix regarding donors (`Channel`), and transfer the gender attribute to the resulting pseudobulks:

In [ ]:
pseudo = pg.pseudobulk(data, 'Channel', 'gender')
pseudo

For details on `pseudobulk` function, please see its [documentation](https://pegasus.readthedocs.io/en/stable/api/pegasus.pseudobulk.html).

## Differential Expression (DE) Analysis on Pseudobulk Matrix

Pegasus has `deseq2` function to perform DE analysis on Pseudobulk data, which is a Python wrapper of [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) package in R (You need to first install the original R package). The code below analyzes based on a regression model considering the gender attribute, and estimates the contrast between `female` and `male`:

In [ ]:
pg.deseq2(pseudo, '~gender', ('gender', 'female', 'male'))

The DE result is a Pandas DataFrame object stored in `pseudo.varm["deseq2"]`, which could be viewed as the following (ranked by log2 fold-change):

In [ ]:
pd.DataFrame(pseudo.varm["deseq2"], index=pseudo.var_names)

Notice that the fold-change calculated has `female` be the numerator, while `male` be the denominator.

For details on `deseq2` function, please see its [documentation](https://pegasus.readthedocs.io/en/stable/api/pegasus.deseq2.html).

### Get significant DE genes in human-readable format

In [ ]:
markers = pg.pseudo.markers(pseudo)
print(markers)

### Write DE results to spreadsheet

In [ ]:
pg.pseudo.write_results_to_excel(markers, 'test.de.xlsx')

### Generate volcano plot

In [ ]:
pg.pseudo.volcano(pseudo)

## Gene Set Enrichment Analysis (GSEA)

Pegasus has `fgsea` function to perform GSEA analysis, which is a Python wrapper of [fgsea](http://bioconductor.org/packages/release/bioc/html/fgsea.html) package in R (You need to first install the original R package):

In [ ]:
pg.fgsea(pseudo, 'log2FoldChange', 'canonical_pathways', 'deseq2', fgsea_key = 'fgsea_deseq2')

The code above runs GSEA analysis based on log2 fold-change values, and use one of the preset gene sets that Pegasus provides:
* `canonical_pathways`: The [MsigDB](http://www.gsea-msigdb.org/gsea/msigdb/collections.jsp) C2/CP gene set.
* `hallmark`: The [MsigDB](http://www.gsea-msigdb.org/gsea/msigdb/collections.jsp) H gene set.
Notice that this function is applicable to DE results from both single-cell and pseudobulk data.

For details on `fgsea` function, please see its [documentation](https://pegasus.readthedocs.io/en/stable/api/pegasus.fgsea.html).

### Generate GSEA plots

Finally, generate the GSEA histograms by `plot_gsea` function. In general, there will be 2 panels:
* Top panel shows the up-regulated pathways in red color.
* Bottom panel shows the down-regulated pathways in green color.

In [ ]:
pg.plot_gsea(pseudo, 'fgsea_deseq2')

As shown in the plot above, for this pseudobulk data, there is no up-regulated (i.e. female significant) pathways, while some down-regulated (i.e. male significant) pathways are observed.